# Download data in the region of interest using `Beacon

In [6]:
using Pkg
Pkg.activate("../")
using HTTP
using JSON
using JSON3
using Dates
using PyPlot
const plt = PyPlot
using NCDatasets
using OrderedCollections
include("./DIVAndFairEase.jl")
token = ENV["beaconAPItoken"];

  Activating project at `~/Projects/DIVAnd-FAIR-EASE`


## Map configuration (`cartopy` and directory)

In [7]:
using Conda
Conda.add("cartopy")
using PyCall
ccrs = pyimport("cartopy.crs")
cfeature = pyimport("cartopy.feature")
mpl = pyimport("matplotlib")
mpl.style.use("./fairease.mplstyle")
coast = cfeature.GSHHSFeature(scale="h")
datacrs = ccrs.PlateCarree();
figdir = "../figures/NorthAdriatic"
mkpath(figdir)

[ Info: Running `conda install -y cartopy` in root environment


Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.9.1




"../figures/NorthAdriatic"

In [8]:
datadir = "../data/"
footprintdir = "../Footprint"
mkpath.([datadir, footprintdir])

2-element Vector{String}:
 "../data"
 "../Footprint"

In [9]:
const beacon_services = OrderedDict(
    "Euro-Argo" => "https://beacon-argo.maris.nl",
    "CORA Profile" => "https://beacon-cora-pr.maris.nl",
    "CORA Timeseries" => "https://beacon-cora-ts.maris.nl",
    "EMODnet Chemistry" => "https://beacon-emod-chem.maris.nl",
    "World Ocean Database" => "https://beacon-wod.maris.nl",
    "SeaDataNet CDI TS" => "https://beacon-cdi-ts.maris.nl",
    "CMEMS BGC" => "https://beacon-cmems.maris.nl",
);

In [10]:
datasourcelist = ["Euro-Argo", "World Ocean Database", "SeaDataNet CDI TS", "CORA Profile", "CORA Timeseries"]
@info("Working on $(length(datasourcelist)) data sources")
domain = [12., 18., 43., 46.]
datestart = Dates.Date(1950, 1, 1)
dateend = Dates.Date(2024, 1, 1)
minlon = domain[1]
maxlon = domain[2]
minlat = domain[3]
maxlat = domain[4]
mindepth = 0.
maxdepth = 500.
regionname = "NorthAdriaticSea"

[ Info: Working on 5 data sources


"NorthAdriaticSea"

## Loop on the data sources

In [11]:
for datasource in datasourcelist
    
    @info("Working on $(datasource)")

    parameter1 = "TEMP"
    if datasource == "World Ocean Database"
        parameter1 = "Temperature"
    elseif datasource == "EMODnet Chemistry"  
        parameter1 = "ITS_90_water_temperature"
    elseif datasource == "SeaDataNet CDI TS"
        parameter1 = "ITS-90 water temperature"
    end
    parameter2 = "PSAL"
    
    @info("$(datestart) → $(dateend), $(parameter1)")
    
    # Download the "footprint"
    beaconURL = beacon_services[datasource]
    footprintURL = joinpath(beaconURL, "api/datasets/footprint")
    datasource_name = replace(datasource, " " => "_")
    footprintfile = joinpath(footprintdir, "Footprint_$(datasource_name).json")
    @info("Footprint endpoint: $(footprintURL)")
    @info("Footprint file: $(footprintfile)")
    
    if isfile(footprintfile)
        @info("Footprint file already downloaded")
    else
        @info("Writing Footprint file")
        open(footprintfile, "w") do io
            r = HTTP.request("GET", footprintURL, 
                ["Authorization" => "Bearer $(token)"],
                response_stream=io)
            @info(r.status)
        end
    end
    
    data = JSON.parsefile(footprintfile);
    attributes = data["unique_column_attributes"]
    params = sort(collect(keys(attributes)));
    
    # Check if the parameters is in the list
    parameter1 in params ? @info("Parameter available") : @warn("Parameter not available")
    # @show(params);
    
    # Build query
    query = DIVAndFairEase.prepare_query_new(datasource, parameter1, parameter2, datestart, dateend, 
        mindepth, maxdepth, minlon, maxlon, minlat, maxlat)
    
    # Construct file and figure names
    filename = joinpath(datadir, "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
    figname = "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.jpg"

    # Write the data in netCDF files
    @info("Data will be written in file:\n$(filename)")
    if isfile(filename)
        @info("File already downloaded")
        rm(filename)
    end
    @time open(filename, "w") do io
        r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
            ["Content-type"=> "application/json",
             "Authorization" => "Bearer $(token)"
            ],
            query, 
            response_stream=io);
        @info(r.status)
    end

    @info("NetCDF file size: $(round(filesize(filename)/1000^2, digits=1))M")
    
    # Create plot
    plot_positions(filename, figname, datasource)
end   

[ Info: Working on Euro-Argo
[ Info: 1950-01-01 → 2024-01-01, TEMP
[ Info: Footprint endpoint: https://beacon-argo.maris.nl/api/datasets/footprint
[ Info: Footprint file: ../Footprint/Footprint_Euro-Argo.json
[ Info: Footprint file already downloaded
[ Info: Parameter available
┌ Info: Data will be written in file:
└ ../data/NorthAdriaticSea_Euro-Argo_TEMP_19500101-20240101_0-500m.nc


  1.471442 seconds (77.42 k allocations: 8.200 MiB, 5.73% compilation time)


[ Info: 200
[ Info: NetCDF file size: 2.0M
[ Info: Working on World Ocean Database
[ Info: 1950-01-01 → 2024-01-01, Temperature
[ Info: Footprint endpoint: https://beacon-wod.maris.nl/api/datasets/footprint
[ Info: Footprint file: ../Footprint/Footprint_World_Ocean_Database.json
[ Info: Footprint file already downloaded
[ Info: Parameter available
┌ Info: Data will be written in file:
└ ../data/NorthAdriaticSea_World_Ocean_Database_Temperature_19500101-20240101_0-500m.nc


 25.303499 seconds (81.19 k allocations: 48.092 MiB, 0.05% compilation time)


[ Info: 200
[ Info: NetCDF file size: 45.8M
[ Info: Working on SeaDataNet CDI TS
[ Info: 1950-01-01 → 2024-01-01, ITS-90 water temperature
[ Info: Footprint endpoint: https://beacon-cdi-ts.maris.nl/api/datasets/footprint
[ Info: Footprint file: ../Footprint/Footprint_SeaDataNet_CDI_TS.json
[ Info: Footprint file already downloaded
[ Info: Parameter available
┌ Info: Data will be written in file:
└ ../data/NorthAdriaticSea_SeaDataNet_CDI_TS_ITS-90_water_temperature_19500101-20240101_0-500m.nc


 28.973981 seconds (89.75 k allocations: 53.397 MiB)


[ Info: 200
[ Info: NetCDF file size: 50.2M
[ Info: Working on CORA Profile
[ Info: 1950-01-01 → 2024-01-01, TEMP
[ Info: Footprint endpoint: https://beacon-cora-pr.maris.nl/api/datasets/footprint
[ Info: Footprint file: ../Footprint/Footprint_CORA_Profile.json
[ Info: Footprint file already downloaded
[ Info: Parameter available
[ Info: Working with CORA dataset
┌ Info: Data will be written in file:
└ ../data/NorthAdriaticSea_CORA_Profile_TEMP_19500101-20240101_0-500m.nc


  6.019789 seconds (22.47 k allocations: 16.864 MiB)


[ Info: 200
[ Info: NetCDF file size: 14.4M
[ Info: Working on CORA Timeseries
[ Info: 1950-01-01 → 2024-01-01, TEMP
[ Info: Footprint endpoint: https://beacon-cora-ts.maris.nl/api/datasets/footprint
[ Info: Footprint file: ../Footprint/Footprint_CORA_Timeseries.json
[ Info: Footprint file already downloaded
[ Info: Parameter available
[ Info: Working with CORA dataset
┌ Info: Data will be written in file:
└ ../data/NorthAdriaticSea_CORA_Timeseries_TEMP_19500101-20240101_0-500m.nc


LoadError: HTTP.Exceptions.StatusError(400, "POST", "/api/query", HTTP.Messages.Response:
"""
HTTP/1.1 400 Bad Request
Content-Type: application/json
Vary: Access-Control-Request-Headers
x-beacon-query-id: 937c63aa-6e7a-41b3-9da2-41846d9e118d
X-Powered-By: ARR/3.0
Date: Thu, 26 Sep 2024 16:17:26 GMT
Content-Length: 94
Set-Cookie: ******

[Message Body was streamed]""")

In [7]:
function plot_positions(filename::String, figname::String, datasource::String)
    NCDataset(filename, "r") do nc
        # print(nc)
        lon = nc["LONGITUDE"][:]
        lat = nc["LATITUDE"][:]

        fig = plt.figure()
        ax = plt.subplot(111, projection=ccrs.EuroPP())
        ax.plot(lon, lat, "go", ms=.5, transform=datacrs)
        ax.set_extent(domain)
        ax.add_feature(coast, lw=.25, color=".25")
        gl = ax.gridlines(crs=datacrs, draw_labels=true,
            linewidth=.5, color=".25", linestyle="--", zorder=2)
        gl.top_labels = false
        gl.right_labels = false
        ax.set_title("Temperature observations ($(length(lon))) in $(datasource)")
        plt.savefig(joinpath(figdir, figname))
        plt.close()

    end
    return nothing
end

plot_positions (generic function with 1 method)

In [8]:
datasource = "CORA Timeseries"
datasource_name = replace(datasource, " " => "_")

@info("Working on $(datasource)")

parameter1 = "TEMP"
parameter2 = "PSAL"
    
    
beaconURL = beacon_services[datasource]
    
# Build query
query = DIVAndFairEase.prepare_query_new(datasource, parameter1, parameter2, datestart, dateend, 
        mindepth, maxdepth, minlon, maxlon, minlat, maxlat)
    
filename = joinpath(datadir, "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
figname = "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.jpg"

# Write the data in netCDF files
@info("Data will be written in file:\n$(filename)")
if isfile(filename)
    @info("File already downloaded")
    rm(filename)
end
@time open(filename, "w") do io
    r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
        ["Content-type"=> "application/json",
        "Authorization" => "Bearer $(token)"
        ],
        query, 
        response_stream=io);
    @info(r.status)
end

[ Info: Working on CORA Timeseries
[ Info: Working with CORA dataset
┌ Info: Data will be written in file:
└ ../data/NorthAdriaticSea_CORA_Timeseries_TEMP_19500101-20240101_0-500m.nc
[ Info: File already downloaded


LoadError: HTTP.Exceptions.StatusError(400, "POST", "/api/query", HTTP.Messages.Response:
"""
HTTP/1.1 400 Bad Request
Content-Type: application/json
Vary: Access-Control-Request-Headers
x-beacon-query-id: 1ea1fe5d-b511-423a-94ea-7b43e7d632e6
X-Powered-By: ARR/3.0
Date: Thu, 26 Sep 2024 19:02:32 GMT
Content-Length: 94
Set-Cookie: ******

[Message Body was streamed]""")